# GRO620 - Activité procédurale 1

Dans cette activité, nous allons principalement travailler sur les éléments nécessaires pour capter une image numériquement, les transformations entre repères 2D et 3D, et l'encodage numérique de la couleur.

In [ ]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## Acquisition

### Q1.1

À partir de la figure 2.23 du livre de référence, décrivez en une phrase le rôle de chacune des étapes de la chaîne d'acquisition d'images numériques.

Optics: Lentilles qui redirige les rayons lumineux vers le capteur

Aperture: Ouverture pour laisser passer une certain quantité de lumière

Shutter (Obturateur): Outil laissant passer la lumière pendant un certain temps et impacte la vitesse d'aquisition

Sensor: Capte la lumière et la transforme en électricité

Gain (ISO): Augmentation du signal analogue

A/D: Transformation de valeur analogue à digital

Demosaic: Division en matrice de couleur

Sharpen: Filtration des couleurs dans la matrice

WhiteBalance: Amélioration de la clarté, niveau de lumière

Gamma/Curve: Amélioration des teintes

Compress: Diminution de l'espace requis par les données

# Q1.2

Quelle est la différence entre paramètres intrinsèques et extrinsèques du caméra ? Décrivez chaque type en une phrase.

Intrinsèque: qui dépend, qui est relié à

Extrinsèque: qui est indépendent de, qui n'est pas rataché à

### Q1.3

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

Le capteur de cette caméra a une taille de 30 mm x 15 mm.

Pouvez-vous estimer la distance focale en mm de la lentille de cette caméra à partir de la matrice $K$ ?

In [ ]:
# Réponse ici.
K = np.array([[620.,   0., 1024.],
              [  0., 620.,  512.],
              [  0.,   0.,    1.]])

f = (30/(2*K[0][2]))*K[0][0]

print("f:\n", f)

### Q1.4

Dans le cadre de cet APP, nous considérons les caméras comme étant idéales, c'est-à-dire qu'on peut obtenir leurs caractéristiques intrinsèques et extrinsèques à partir de quelques paramètres seulement.

**a)** Qu'est-ce qui rend les vraies caméras non-idéales ? Nommez des facteurs autant pour les caractéristiques intrinsèques que extrinsèques.

Le positionnement de la caméra pour l'extrinsèque et tout les composantes de la caméra(lentilles, Aperture, Shutter) pour l'intrésèque

**b)** Que doit on faire pour obtenir les caractéristiques d'une caméra non-idéale ?

*(réponse ici)*

### Q1.5

**a)** Pourquoi deux appareils de capture peuvent produire des valeurs RGB différentes d'une même couleur ? 

Le moindre changement dans la chaîne d'aquisition

**b)** Que peut-on faire pour comparer numériquement des couleurs provenant de deux capteurs différents ?

Courbe de couleur

## Repères et coordonnées

### Q2.1

Supposons ces 2 repères :

![](images_doc/proc1-q2_1-frames.png)

**a)** Trouvez la matrice homogène permettant de transformer un point du repère $\{1\}$ au repère $\{0\}$.

In [ ]:
T_10 = np.identity(4) # Génère une matrice identité 4x4
R = np.array([[0,1,0],
              [1,0,0],
              [0,0,-1]])
D = np.array([[240,80,120]])
T_10[:3,:3] = R
T_10[0:3,3] = D
print("T_10:\n", T_10)

**b)** Trouvez maintenant la transformation inverse.

In [ ]:
T_01 = np.identity(4)
T_01 = np.linalg.inv(T_10)
print("T_01\n", T_01)

**c)** Soit le point $p_0 = [8, 5, 1]^T$, un point dans le repère $\{0\}$. Trouvez $p_1$, ses coordonnées dans le repère $\{1\}$.

In [ ]:
p_0 = [8, 5, 1]
p_1 = [0, 0, 0]
p_0 = [8,5,1,1]
p_1 = [0,0,0,0]
p_1 = np.dot(T_01, p_0)
print("p_1:\n", p_1)

### Q2.2

Supposons maintenant que le repère $\{1\}$ représente une caméra avec les caractéristiques intrinsèques $K$ de la question Q1.3.

**a)** Trouvez la matrice de projection P complète permettant de projeter un point $p$ décrit dans le repère $\{0\}$.

In [ ]:
#print(K) # Si vous n'avez pas réutilisé la variable K, elle aura toujours la même valeur qu'à la question Q1.3.
K_p = np.identity(4)
K_p[:3, :3] = K
P = np.dot(K_p,T_01)
print("P:\n", P)

**b)** Soit le point $p_0 = [0.250, 0.010, 0.000]$. Trouvez le point $x_s$, les coordonnées du point $p_0$ perçu par la caméra.

In [ ]:
p_0 = np.array([0.250, 0.010, 0.000, 1])
x_s = np.dot(P, p_0)
x_s = x_s / x_s[2] #forme normalisé
print("x_s:\n", x_s)

## Reprojection 2D à 3D

### Q3.1

Supposons que le plan XY du repère $\{0\}$ est un convoyeur. Quelle serait sa largeur maximale (mesurée sur l'axe Y) si on souhaite que la caméra la capte au complet dans son image ? 

In [ ]:
l_conv = 0
l_conv = ((1024 * 15/2) / f )
print(l_conv)

### Q3.2

Soit le point $x_s = [120, 200]$, un point dans l'image perçu par la caméra décrite plus haut. On suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Trouvez les coordonnées du point $p_0$ qui correspond à ce même point dans le repère $\{0\}$.

In [ ]:
x_s_1 = x_s /x_s[3]
P_t = np.linalg.inv(P)
p_0_1 = np.dot(P_t, x_s_1)
print("Cas1:\n",p_0_1)
xs_2 = np.array([120,200])
d = 1/120 #distance en mm selon l'axe z
x_s2 = np.array([120,200,1,d])
x_s_2 = x_s2/x_s2[3]
p_0_2 = np.dot(P_t, x_s_2)
print("Cas2:\n",p_0_2)